Ноздрова В.
ИУ5-81Б


#Задача 1

In [ ]:
Variant<-15
set.seed(Variant)
m<-sample(c(6:20),1)
lambda<-runif(1,0.1, 2)
Q<-rexp(m,0.3)
q<-sample(c(1:4),1)
View(data.frame(m,q, lambda))
print(Q)

m,q,lambda
<int>,<int>,<dbl>
10,1,0.4705835


 [1] 0.84811616 0.58679153 2.20952680 9.18050457 0.97433940 0.06060261
 [7] 1.37752378 1.03381451 0.32621141 2.02484933


Среднее время обслуживания программ:

$T_{сист} = \frac{1}{\mu-\lambda}$

In [ ]:
lambda/min(Q)

[1] 7.765069

По варианту $\frac{\lambda}{min(Q)} > 1$, поэтому значение $\lambda$ необходимо изменить

In [ ]:
set.seed(Variant)
lambda <- min(Q) - 0.05; lambda

[1] 0.01060261

$T_{\text{сист}}=\frac{1}{\mu\left(1-\frac{\lambda}{\mu}\right)}$

In [ ]:
mu <- 1/(sum(Q)/length(Q))
Tt_sist <- 1/(mu*(1-(lambda/mu)))
Tt_sist

[1] 1.899737

Экспериментально:

In [ ]:
install.packages("simmer")
library("simmer")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
SIM_TIME = 5000000

program <- trajectory("program's path") %>%
   seize("server") %>%
   timeout(function() sample(Q,1)) %>%
   release("server")

distr <- function() rexp(1, lambda)
syst <- simmer("syst") %>%
   add_resource("server", mon=T) %>%
   add_generator("prog", program, from(distr, distr), mon=2)

syst %>% run(until = SIM_TIME) %>% invisible

stats <- get_mon_arrivals(syst, per_resource = T)
Te_sist <- mean(stats$end_time - stats$start_time)

Результат:

In [ ]:
param <- "T_sist"
teor <- Tt_sist
exper <- Te_sist
data.frame(param,teor,exper)

param,teor,exper
<chr>,<dbl>,<dbl>
T_sist,1.899737,1.906877


SPT:

In [ ]:
Qs <- sort(Q)

In [ ]:
SIM_TIME = 5000000

distr <- function() rexp(1, lambda)

init <- trajectory("init")%>%
        timeout(distr) %>%
        activate(function() {x<-get_sources(spt); x[sample(2:length(get_sources(spt)),1)]}) %>%
        rollback(3, Inf)

spt_prog <- trajectory("program's path") %>%
   seize("server") %>%
   timeout_from_attribute("timeout") %>%
   release("server")

spt <- simmer("spt") %>%
   add_resource("server", mon=T, preemptive = T, preempt_order = "fifo", queue_priority = 1) %>%
   add_generator("init", init, at(0))%>%invisible
   for(i in 1:m) spt %>%
     add_generator(paste("pr",i), join(trajectory()%>%
                                       set_attribute("timeout",Qs[i]),
                                       spt_prog), when_activated(1), mon=2, priority=i) %>% invisible

spt %>% run(until = SIM_TIME) %>% invisible

spt_stats <- get_mon_arrivals(spt, per_resource = T)
spt_sist <- mean(spt_stats$end_time - spt_stats$start_time);spt_sist

[1] 1.821439

Round Robin:

In [ ]:
RoundRobin <- function(q)

{
ST <- 100000

distr <- function() rexp(1, lambda)

init1 <- trajectory("init")%>%
        timeout(distr) %>%
        activate(function() {x<-get_sources(rr); x[sample(2:length(get_sources(rr)),1)]}) %>%
        rollback(3, Inf)

rr_prog <- trajectory("program's path") %>%
   set_attribute("time",function() sample(Qs,1))%>%
   seize("server") %>%
   timeout(q)%>%
   release("server")%>%
   set_attribute("time",-q,"+")%>%
   branch(function() ifelse(get_attribute(rr,"time")>0,1,0),T,
          trajectory()%>%rollback(5,Inf))

rr <- simmer("rr") %>%
   add_resource("server", mon=T, preemptive = T, preempt_order = "fifo", queue_priority = 1) %>%
   add_generator("init", init1, at(0))%>%invisible
   for(i in 1:m) rr %>%
     add_generator(paste("pr",i," "), rr_prog, when_activated(1), mon=2) %>% invisible

rr %>% run(until = ST) %>% invisible

rr_stats <- get_mon_arrivals(rr, per_resource = T)
view <- data.frame(rr_stats$name,rr_stats$end_time-rr_stats$start_time)
view <-  aggregate(. ~ rr_stats.name, view, sum, na.rm = TRUE)
return(mean(view$rr_stats.end_time...rr_stats.start_time))}

In [ ]:
rq <-RoundRobin(q)

Результаты:

In [ ]:
df <- data.frame("T_sist", Te_sist, spt_sist, rq)
colnames(df) <-c("param", "exper", "spt", "rr(q=1)")
df

param,exper,spt,rr(q=1)
<chr>,<dbl>,<dbl>,<dbl>
T_sist,1.906877,1.821439,2.429563


Сравнение при различных q:

In [ ]:
min(Q)

[1] 0.06060261

In [ ]:
dfr <- data.frame("T_sist", RoundRobin(0.04), RoundRobin(0.05), RoundRobin(min(Q)), RoundRobin(0.1), RoundRobin(0.2), RoundRobin(0.5),rq)
colnames(dfr) <-c("param", "q=0.04", "q=0.05", "q=min(mu)", "q=0.1", "q=0.2", "q=0.5", "q=1")
dfr

param,q=0.04,q=0.05,q=min(mu),q=0.1,q=0.2,q=0.5,q=1
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
T_sist,1.881127,1.924829,1.847771,1.968982,1.943333,2.110401,2.429563
